In [ ]:
input_test = """029A
980A
179A
456A
379A"""

In [ ]:
import numpy as np
from collections import defaultdict

In [ ]:
#uncertain: ordering of doubles
#ld is better than dl; dl requires two groups of lefts (tested for 10 downs, but could in theory interact with others)
#ur seems better than ru; this is kind of weird?
#found best doubles-ordering by testing
dict_map = {"A" : "A", "uA" : "lArA", "rA": "dAuA", "dA": "ldAurA", "lA": "dllArruA"} #all simples
dict_map["ldA"] = "dllArAurA"
dict_map["luA"] = "dllAruArA"
dict_map["ruA"] = "dAluArA"
dict_map["rdA"] = "lAdAurA"
dict_map["dlA"] = "ldAlArruA"
dict_map["drA"] = "ldArAuA"
dict_map["ulA"] = "lAdlArruA"
dict_map["urA"] = "lAdrAuA"

def pass_down_group(g):
    if g in dict_map: return dict_map[g]
    else:
        g_base = ""
        g_extras = ""
        c_prev = None
        for c in g:
            if c == c_prev: g_extras += "A"
            else:
                g_base += c
                c_prev = c
        return dict_map[g_base] + g_extras

In [ ]:
def pass_down_group_dict(gd):
    new_group_dict = defaultdict(int)
    for g,c in gd.items():
        g_to_seq = pass_down_group(g)
        groups_to = [g_to + "A" for g_to in g_to_seq.split('A')][:-1]
        for g_to in groups_to:
            new_group_dict[g_to] += c
    return new_group_dict

In [ ]:
def pass_down_group_dict_r(gd, r):
    for i in range(r): gd = pass_down_group_dict(gd)
    return gd

In [ ]:
udlr_map = {"u" : np.array((-1,0)), "d" : np.array((1,0)), "l" : np.array((0,-1)), "r" : np.array((0,1)) }
kpad = np.array([[7,8,9],[4,5,6],[1,2,3,],["X",0,"A"]])

def validate_move(pos_from, pos_avoid, move):
    pos = pos_from.copy()
    for c in move:
        pos += udlr_map[c]
        if np.all(pos == pos_avoid): return False
    return True



def get_group_dict_from_seq(seq):
    gd = defaultdict(int)
    
    pos_from = np.argwhere(kpad == "A")[0]
    pos_avoid = np.argwhere(kpad == "X")[0]

    for char in seq:
        pos_to = np.argwhere(kpad == char)[0]
        move = pos_to - pos_from
        v,h = int(move[0]),int(move[1])
        v_str = "d" * v if v > 0 else "u" * -v
        h_str = "r" * h if h > 0 else "l" * -h
        vh,hv = v_str + h_str, h_str + v_str

        moves = set([m for m in [vh, hv] if validate_move(pos_from, pos_avoid, m)])
        #Choose correct ordering of vertical/horizontal gourps if both are possible
        if v > 0 and h > 0: move = vh
        if v > 0 and h < 0: move = hv
        if v < 0 and h > 0: move = vh
        if v < 0 and h < 0: move = hv
        #Get the only possible move if either move failed validation
        if len(moves) == 1: move = list(moves)[0]
            
        gd[move + "A"] += 1
        pos_from = pos_to
    return gd

def get_complexity_from_sequence(seq, r=2):
    group_dict = get_group_dict_from_seq(seq)
    group_dict = pass_down_group_dict_r(group_dict, r)
    return sum([len(g)*c for g,c in group_dict.items()])

def solve(inp, r=2):
    tot = 0
    for seq in inp.split('\n'):
        n = int(seq[:-1])
        tot += n * get_complexity_from_sequence(seq, r)
    return tot

In [ ]:
solve(input_full, 25)

In [ ]:
input_full = """540A
839A
682A
826A
974A"""